<a href="https://colab.research.google.com/github/ritwiks9635/My_Neural_Network_Architecture/blob/main/Gradient_Centralization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Gradient Centralization for Better Training Performance**

## Introduction

This example implements [Gradient Centralization](https://arxiv.org/abs/2004.01461), a
new optimization technique for Deep Neural Networks by Yong et al., and demonstrates it
on Laurence Moroney's [Horses or Humans
Dataset](https://www.tensorflow.org/datasets/catalog/horses_or_humans). Gradient
Centralization can both speedup training process and improve the final generalization
performance of DNNs. It operates directly on gradients by centralizing the gradient
vectors to have zero mean. Gradient Centralization morever improves the Lipschitzness of
the loss function and its gradient so that the training process becomes more efficient
and stable.

In [1]:
from time import time
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

In [2]:
num_classes = 2
input_shape = (300, 300, 3)
dataset_name = "horses_or_humans"
batch_size = 128
AUTOTUNE = tf.data.AUTOTUNE

(train_ds, test_ds), metadata = tfds.load(
        name = dataset_name,
        split = [tfds.Split.TRAIN, tfds.Split.TEST],
        with_info = True,
        as_supervised = True)


print(f"Image shape: {metadata.features['image'].shape}")
print(f"Training images: {metadata.splits['train'].num_examples}")
print(f"Test images: {metadata.splits['test'].num_examples}")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1027 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteTWSJE5/horses_or_humans-train.tfrecord*..…

Generating test examples...:   0%|          | 0/256 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteTWSJE5/horses_or_humans-test.tfrecord*...…

Dataset horses_or_humans downloaded and prepared to /root/tensorflow_datasets/horses_or_humans/3.0.0. Subsequent calls will reuse this data.
Image shape: (300, 300, 3)
Training images: 1027
Test images: 256


In [3]:
rescale = layers.Rescaling(1.0 / 255)

data_augmentation = tf.keras.Sequential(
    [
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.3),
        layers.RandomZoom(0.2),
    ])


def prepare(ds, shuffle=False, augment=False):
    # Rescale dataset
    ds = ds.map(lambda x, y: (rescale(x), y), num_parallel_calls=AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(1024)

    # Batch dataset
    ds = ds.batch(batch_size)

    # Use data augmentation only on the training set
    if augment:
        ds = ds.map(
            lambda x, y: (data_augmentation(x, training=True), y),
            num_parallel_calls=AUTOTUNE)

    # Use buffered prefecting
    return ds.prefetch(buffer_size=AUTOTUNE)

In [4]:
train_ds = prepare(train_ds, shuffle = True, augment = True)
test_ds = prepare(test_ds)

In [5]:
model = tf.keras.Sequential(
    [
        layers.Conv2D(16, (3,3), activation = "relu", input_shape  = input_shape),
        layers.MaxPool2D(2,2),
        layers.Conv2D(32, (3,3), activation = "relu"),
        layers.Dropout(0.5),
        layers.MaxPool2D(2,2),
        layers.Conv2D(64, (3,3), activation = "relu"),
        layers.Dropout(0.5),
        layers.MaxPool2D(2,2),
        layers.Conv2D(64, (3,3), activation = "relu"),
        layers.MaxPool2D(2,2),
        layers.Conv2D(64, (3,3), activation = "relu"),
        layers.MaxPool2D(2,2),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(512, activation = "relu"),
        layers.Dense(1, activation = "sigmoid")
    ])

## Implement Gradient Centralization

We will now
subclass the `RMSProp` optimizer class modifying the
`keras.optimizers.Optimizer.get_gradients()` method where we now implement Gradient
Centralization. On a high level the idea is that let us say we obtain our gradients
through back propogation for a Dense or Convolution layer we then compute the mean of the
column vectors of the weight matrix, and then remove the mean from each column vector.

The experiments in [this paper](https://arxiv.org/abs/2004.01461) on various
applications, including general image classification, fine-grained image classification,
detection and segmentation and Person ReID demonstrate that GC can consistently improve
the performance of DNN learning.

Also, for simplicity at the moment we are not implementing gradient cliiping functionality,
however this quite easy to implement.

At the moment we are just creating a subclass for the `RMSProp` optimizer
however you could easily reproduce this for any other optimizer or on a custom
optimizer in the same way. We will be using this class in the later section when
we train a model with Gradient Centralization.

In [6]:
class GCRMSprop(RMSprop):
    def get_gradients(self, loss, params):
        # We here just provide a modified get_gradients() function since we are
        # trying to just compute the centralized gradients.

        grads = []
        gradients = super().get_gradients()
        for grad in gradients:
            grad_len = len(grad.shape)
            if grad_len > 1:
                axis = list(range(grad_len - 1))
                grad -= tf.reduce_mean(grad, axis=axis, keep_dims=True)
            grads.append(grad)

        return grads


optimizer = GCRMSprop(learning_rate=1e-4)

## Training utilities

We will also create a callback which allows us to easily measure the total training time
and the time taken for each epoch since we are interested in comparing the effect of
Gradient Centralization on the model we built above.

In [7]:
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time() - self.epoch_time_start)


## Train the model without GC

We now train the model we built earlier without Gradient Centralization which we can
compare to the training performance of the model trained with Gradient Centralization.

In [8]:
time_callback_no_gc = TimeHistory()
model.compile(
    loss="binary_crossentropy",
    optimizer=RMSprop(learning_rate=1e-4),
    metrics=["accuracy"])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 149, 149, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 dropout (Dropout)           (None, 147, 147, 32)      0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 73, 73, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)       

We also save the history since we later want to compare our model trained with and not
trained with Gradient Centralization

In [9]:
history_no_gc = model.fit(
    train_ds, epochs=10, verbose=1, callbacks=[time_callback_no_gc])

Epoch 1/10
9/9 [==============================] - 34s 1s/step - loss: 0.7427 - accuracy: 0.5024
Epoch 2/10
9/9 [==============================] - 19s 2s/step - loss: 0.6891 - accuracy: 0.5472
Epoch 3/10
9/9 [==============================] - 19s 2s/step - loss: 0.6722 - accuracy: 0.5823
Epoch 4/10
9/9 [==============================] - 19s 2s/step - loss: 0.6659 - accuracy: 0.6105
Epoch 5/10
9/9 [==============================] - 19s 2s/step - loss: 0.6470 - accuracy: 0.6222
Epoch 6/10
9/9 [==============================] - 20s 2s/step - loss: 0.6082 - accuracy: 0.7108
Epoch 7/10
9/9 [==============================] - 19s 2s/step - loss: 0.5953 - accuracy: 0.6923
Epoch 8/10
9/9 [==============================] - 24s 2s/step - loss: 0.5754 - accuracy: 0.7517
Epoch 9/10
9/9 [==============================] - 19s 2s/step - loss: 0.5429 - accuracy: 0.7498
Epoch 10/10
9/9 [==============================] - 24s 2s/step - loss: 0.5079 - accuracy: 0.7790


## Train the model with GC

We will now train the same model, this time using Gradient Centralization,
notice our optimizer is the one using Gradient Centralization this time.

In [11]:
time_callback_gc = TimeHistory()
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])



history_gc = model.fit(train_ds, epochs=10, verbose=1, callbacks=[time_callback_gc])

Epoch 1/10
9/9 [==============================] - 25s 2s/step - loss: 0.5474 - accuracy: 0.7254
Epoch 2/10
9/9 [==============================] - 21s 2s/step - loss: 0.4771 - accuracy: 0.7955
Epoch 3/10
9/9 [==============================] - 20s 2s/step - loss: 0.4639 - accuracy: 0.8082
Epoch 4/10
9/9 [==============================] - 19s 2s/step - loss: 0.4308 - accuracy: 0.8082
Epoch 5/10
9/9 [==============================] - 21s 2s/step - loss: 0.4787 - accuracy: 0.7936
Epoch 6/10
9/9 [==============================] - 21s 2s/step - loss: 0.3926 - accuracy: 0.8315
Epoch 7/10
9/9 [==============================] - 19s 2s/step - loss: 0.4077 - accuracy: 0.8354
Epoch 8/10
9/9 [==============================] - 20s 2s/step - loss: 0.4162 - accuracy: 0.8189
Epoch 9/10
9/9 [==============================] - 19s 2s/step - loss: 0.3880 - accuracy: 0.8452
Epoch 10/10
9/9 [==============================] - 19s 2s/step - loss: 0.3767 - accuracy: 0.8384


## Comparing performance

In [12]:
print("Not using Gradient Centralization")
print(f"Loss: {history_no_gc.history['loss'][-1]}")
print(f"Accuracy: {history_no_gc.history['accuracy'][-1]}")
print(f"Training Time: {sum(time_callback_no_gc.times)}")

print("Using Gradient Centralization")
print(f"Loss: {history_gc.history['loss'][-1]}")
print(f"Accuracy: {history_gc.history['accuracy'][-1]}")
print(f"Training Time: {sum(time_callback_gc.times)}")

Not using Gradient Centralization
Loss: 0.5078917145729065
Accuracy: 0.7789678573608398
Training Time: 215.31202745437622
Using Gradient Centralization
Loss: 0.37669432163238525
Accuracy: 0.8383641839027405
Training Time: 203.80182600021362
